# Механизмы внимания и трансформеры

Одним из основных недостатков рекуррентных сетей является то, что все слова в последовательности оказывают одинаковое влияние на результат. Это приводит к недостаточной производительности стандартных моделей LSTM encoder-decoder для задач преобразования последовательностей, таких как распознавание именованных сущностей и машинный перевод. На практике отдельные слова во входной последовательности часто оказывают большее влияние на выходные данные, чем другие.

Рассмотрим модель преобразования последовательностей, например, машинный перевод. Она реализуется с помощью двух рекуррентных сетей, где одна сеть (**encoder**) преобразует входную последовательность в скрытое состояние, а другая (**decoder**) разворачивает это скрытое состояние в переведенный результат. Проблема такого подхода заключается в том, что конечному состоянию сети сложно запомнить начало предложения, что ухудшает качество модели при обработке длинных предложений.

**Механизмы внимания** предоставляют способ взвешивания контекстного влияния каждого входного вектора на каждое предсказание выхода RNN. Это реализуется путем создания "ярлыков" между промежуточными состояниями входной RNN и выходной RNN. Таким образом, при генерации выходного символа $y_t$ мы учитываем все скрытые состояния входа $h_i$ с различными весовыми коэффициентами $\alpha_{t,i}$.

![Изображение, показывающее модель encoder/decoder с аддитивным слоем внимания](../../../../../translated_images/ru/encoder-decoder-attention.7a726296894fb567.webp)
*Модель encoder-decoder с механизмом аддитивного внимания из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), цитируется из [этого блога](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

Матрица внимания $\{\alpha_{i,j}\}$ представляет степень, в которой определенные слова входной последовательности участвуют в генерации конкретного слова выходной последовательности. Ниже приведен пример такой матрицы:

![Изображение, показывающее пример выравнивания, найденного RNNsearch-50, взято из Bahdanau - arviz.org](../../../../../translated_images/ru/bahdanau-fig3.09ba2d37f202a6af.webp)

*Рисунок взят из [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Рис.3)*

Механизмы внимания являются основой многих современных или близких к современным достижений в области обработки естественного языка. Однако добавление внимания значительно увеличивает количество параметров модели, что приводит к проблемам масштабирования с RNN. Основным ограничением масштабирования RNN является то, что рекуррентная природа моделей затрудняет пакетную обработку и параллелизацию обучения. В RNN каждый элемент последовательности должен обрабатываться в последовательном порядке, что делает параллелизацию сложной.

Применение механизмов внимания в сочетании с этим ограничением привело к созданию современных моделей трансформеров, которые мы знаем и используем сегодня, таких как BERT и OpenGPT3.

## Модели трансформеров

Вместо передачи контекста каждого предыдущего предсказания в следующий шаг оценки, **модели трансформеров** используют **позиционные кодировки** и **внимание**, чтобы захватить контекст заданного входа в пределах предоставленного окна текста. На изображении ниже показано, как позиционные кодировки с вниманием могут захватывать контекст в пределах заданного окна.

![Анимированный GIF, показывающий, как выполняются оценки в моделях трансформеров.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Поскольку каждая входная позиция сопоставляется независимо с каждой выходной позицией, трансформеры могут лучше параллелизоваться, чем RNN, что позволяет создавать гораздо более крупные и выразительные языковые модели. Каждая "голова внимания" может использоваться для изучения различных отношений между словами, что улучшает задачи обработки естественного языка.

## Создание простой модели трансформера

Keras не содержит встроенного слоя трансформера, но мы можем создать его самостоятельно. Как и раньше, мы сосредоточимся на классификации текста из набора данных AG News, но стоит отметить, что модели трансформеров показывают лучшие результаты в более сложных задачах обработки естественного языка.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Новые слои в Keras должны быть подклассами класса `Layer` и реализовывать метод `call`. Давайте начнем с слоя **Positional Embedding**. Мы будем использовать [некоторый код из официальной документации Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Мы будем предполагать, что все входные последовательности дополнены до длины `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Этот слой состоит из двух слоев `Embedding`: для встраивания токенов (как мы обсуждали ранее) и для встраивания позиций токенов. Позиции токенов создаются как последовательность натуральных чисел от 0 до `maxlen` с использованием `tf.range`, а затем передаются через слой встраивания. Два полученных вектора встраивания складываются, создавая позиционно-встроенное представление входных данных формы `maxlen`$\times$`embed_dim`.

Теперь давайте реализуем блок трансформера. Он будет принимать выходные данные ранее определенного слоя встраивания:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Теперь мы готовы определить полную модель трансформера:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Модели трансформеров BERT

**BERT** (Bidirectional Encoder Representations from Transformers) — это очень большая многослойная сеть трансформеров с 12 слоями для *BERT-base* и 24 для *BERT-large*. Модель сначала проходит предварительное обучение на большом корпусе текстовых данных (WikiPedia + книги) с использованием обучения без учителя (предсказание замаскированных слов в предложении). Во время предварительного обучения модель усваивает значительный уровень понимания языка, который затем можно использовать с другими наборами данных с помощью тонкой настройки. Этот процесс называется **обучением переноса**.

![изображение с http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ru/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

Существует множество вариаций архитектур трансформеров, включая BERT, DistilBERT, BigBird, OpenGPT3 и другие, которые можно тонко настраивать.

Давайте посмотрим, как мы можем использовать предварительно обученную модель BERT для решения нашей традиционной задачи классификации последовательностей. Мы заимствуем идею и часть кода из [официальной документации](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Для загрузки предварительно обученных моделей мы будем использовать **Tensorflow hub**. Сначала загрузим векторизатор, специфичный для BERT:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Важно использовать тот же векторизатор, который использовался для обучения оригинальной сети. Кроме того, векторизатор BERT возвращает три компонента:
* `input_word_ids` — последовательность номеров токенов для входного предложения
* `input_mask` — показывает, какая часть последовательности содержит фактический ввод, а какая является заполнением. Это похоже на маску, создаваемую слоем `Masking`
* `input_type_ids` используется для задач языкового моделирования и позволяет указать два входных предложения в одной последовательности.

Затем мы можем создать экземпляр извлекателя признаков BERT:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Итак, слой BERT возвращает несколько полезных результатов:
* `pooled_output` — это результат усреднения всех токенов в последовательности. Его можно рассматривать как интеллектуальное семантическое представление всей сети. Это эквивалент вывода слоя `GlobalAveragePooling1D` в нашей предыдущей модели.
* `sequence_output` — это вывод последнего слоя трансформера (соответствует выводу `TransformerBlock` в нашей модели выше).
* `encoder_outputs` — это выводы всех слоев трансформера. Поскольку мы загрузили 4-слойную модель BERT (как вы, вероятно, догадались из названия, содержащего `4_H`), она имеет 4 тензора. Последний из них совпадает с `sequence_output`.

Теперь мы определим сквозную модель классификации. Мы будем использовать *функциональное определение модели*, когда задаем входные данные модели, а затем предоставляем серию выражений для вычисления ее вывода. Мы также сделаем веса модели BERT не обучаемыми и будем тренировать только финальный классификатор:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Несмотря на то, что обучаемых параметров мало, процесс идет довольно медленно, так как извлечение признаков с помощью BERT требует больших вычислительных ресурсов. Похоже, нам не удалось достичь приемлемой точности, либо из-за недостаточного обучения, либо из-за нехватки параметров модели.

Давайте попробуем разморозить веса BERT и обучить их тоже. Для этого потребуется очень маленькая скорость обучения, а также более тщательная стратегия обучения с использованием **разогрева** и оптимизатора **AdamW**. Мы будем использовать пакет `tf-models-official` для создания оптимизатора:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Как вы можете заметить, обучение идет довольно медленно — но вы можете поэкспериментировать и обучить модель на несколько эпох (5-10), чтобы увидеть, сможете ли вы добиться лучшего результата по сравнению с подходами, которые мы использовали ранее.

## Библиотека Huggingface Transformers

Еще один очень распространенный (и немного более простой) способ использования моделей Transformer — это [пакет HuggingFace](https://github.com/huggingface/), который предоставляет простые строительные блоки для различных задач обработки естественного языка. Он доступен как для Tensorflow, так и для PyTorch — еще одной очень популярной библиотеки для работы с нейронными сетями.

> **Примечание**: Если вам неинтересно разбираться, как работает библиотека Transformers, вы можете пропустить конец этой тетрадки, так как вы не увидите ничего принципиально нового по сравнению с тем, что мы делали выше. Мы будем повторять те же шаги по обучению модели BERT, используя другую библиотеку и существенно более крупную модель. Таким образом, процесс включает довольно длительное обучение, поэтому вы можете просто просмотреть код.

Давайте посмотрим, как наша задача может быть решена с помощью [Huggingface Transformers](http://huggingface.co).


Первым делом нам нужно выбрать модель, которую мы будем использовать. Помимо встроенных моделей, Huggingface содержит [онлайн-репозиторий моделей](https://huggingface.co/models), где можно найти множество предварительно обученных моделей, созданных сообществом. Все эти модели можно загрузить и использовать, просто указав имя модели. Все необходимые бинарные файлы для модели будут автоматически загружены.

Иногда вам может понадобиться загрузить собственные модели. В таком случае вы можете указать директорию, содержащую все необходимые файлы, включая параметры для токенизатора, файл `config.json` с параметрами модели, бинарные веса и т.д.

На основе имени модели мы можем создать как саму модель, так и токенизатор. Давайте начнем с токенизатора:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

Объект `tokenizer` содержит функцию `encode`, которую можно напрямую использовать для кодирования текста:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Мы также можем использовать токенизатор для кодирования последовательности таким образом, чтобы она была подходящей для передачи модели, то есть включая поля `token_ids`, `input_mask` и т. д. Мы также можем указать, что хотим использовать тензоры Tensorflow, предоставив аргумент `return_tensors='tf'`:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

В нашем случае мы будем использовать предварительно обученную модель BERT под названием `bert-base-uncased`. *Uncased* означает, что модель нечувствительна к регистру.

При обучении модели нам необходимо предоставить токенизированную последовательность в качестве входных данных, поэтому мы разработаем конвейер обработки данных. Поскольку `tokenizer.encode` является функцией на Python, мы будем использовать тот же подход, что и в последнем разделе, вызывая её с помощью `py_function`:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Теперь мы можем загрузить фактическую модель, используя пакет `BertForSequenceClassification`. Это гарантирует, что наша модель уже имеет необходимую архитектуру для классификации, включая финальный классификатор. Вы увидите предупреждающее сообщение, указывающее, что веса финального классификатора не инициализированы, и модель потребует предварительного обучения - это совершенно нормально, потому что именно это мы собираемся сделать!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Как видно из `summary()`, модель содержит почти 110 миллионов параметров! Предположительно, если мы хотим выполнить простую задачу классификации на относительно небольшом наборе данных, мы не хотим обучать базовый слой BERT:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Теперь мы готовы начать обучение!

> **Примечание**: Обучение полноразмерной модели BERT может занять очень много времени! Поэтому мы будем обучать её только на первых 32 батчах. Это сделано лишь для демонстрации процесса настройки обучения модели. Если вы хотите попробовать обучение в полном масштабе, просто удалите параметры `steps_per_epoch` и `validation_steps` и приготовьтесь ждать!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Если увеличить количество итераций, подождать достаточно долго и обучить модель на нескольких эпохах, можно ожидать, что классификация с использованием BERT даст наилучшую точность! Это связано с тем, что BERT уже довольно хорошо понимает структуру языка, и нам нужно лишь дообучить финальный классификатор. Однако, поскольку BERT — это большая модель, весь процесс обучения занимает много времени и требует значительных вычислительных ресурсов! (GPU, и желательно больше одного).

> **Note:** В нашем примере мы использовали одну из самых маленьких предобученных моделей BERT. Существуют более крупные модели, которые, вероятно, дадут лучшие результаты.


## Основные выводы

В этом разделе мы рассмотрели современные архитектуры моделей, основанные на **трансформерах**. Мы применили их для задачи классификации текста, но аналогично модели BERT могут использоваться для извлечения сущностей, ответа на вопросы и других задач обработки естественного языка.

Модели на основе трансформеров представляют собой передовой уровень в NLP, и в большинстве случаев они должны быть первым решением, с которого вы начинаете эксперименты при разработке пользовательских NLP-решений. Однако понимание базовых принципов рекуррентных нейронных сетей, обсуждаемых в этом модуле, крайне важно, если вы хотите создавать продвинутые нейронные модели.



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, имейте в виду, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
